In [6]:
# import libraries
import sys, os, signal, time, threading, json, webbrowser

import jupyros
import rospy, message_filters, actionlib
from std_msgs.msg import String, Header
from actionlib_msgs.msg import GoalStatus

! source /home/qcr/cgras_moveit_ws/devel/setup.bash
sys.path.append('/home/qcr/cgras_moveit_ws/devel/lib/python3/dist-packages')

# project modules
from cgras_robot.msg import RobotCommandAction, RobotCommandGoal

rospy.init_node('action_node')

In [ ]:
jupyros.action_client('/cgras/robot/do', RobotCommandAction, RobotCommandGoal, callbacks={})

In [62]:
def cb_calibrate_done(status, result):
    if status == GoalStatus.SUCCEEDED:
        print(f'success: the location apriltag found {result.data}')
    elif status == GoalStatus.ABORTED:
        print(f'error: the error is {result.data}')   

def cb_move_done(status, result):   
    if status == GoalStatus.SUCCEEDED:
        print(f'success: moved to the target')
    elif status == GoalStatus.ABORTED:
        print(f'error: the error is {result.data}')   

def cb_reset_done(status, result): 
    if status == GoalStatus.SUCCEEDED:
        print(f'success: moved to the desired pose')
    elif status == GoalStatus.ABORTED:
        print(f'error: the error is {result.data}')             

# -- callback from the GUI console
def call_robot_to_calibrate(*args):
    action_client = actionlib.SimpleActionClient('/cgras/robot/do', RobotCommandAction)
    action_client.wait_for_server()
    goal = RobotCommandGoal(target=[RobotCommandGoal.ACTION_CALIBRATE])
    action_client.send_goal(goal, done_cb=cb_calibrate_done)
            
def call_robot_to_move_cell(*args):
    action_client = actionlib.SimpleActionClient('/cgras/robot/do', RobotCommandAction)
    action_client.wait_for_server()
    if len(args) == 0:
        target = [RobotCommandGoal.ACTION_MOVE_CELL, 0, 0, 0, 0]
    else:
        target = [RobotCommandGoal.ACTION_MOVE_CELL] + args
    goal = RobotCommandGoal(target=target)
    action_client.send_goal(goal, done_cb=cb_move_done)

def call_robot_to_move_posename(*args):
    action_client = actionlib.SimpleActionClient('/cgras/robot/do', RobotCommandAction)
    action_client.wait_for_server()
    if len(args) == 0:
        target = [RobotCommandGoal.ACTION_MOVE_NAME, RobotCommandGoal.POSENAME_STOW]
    else:
        target = [RobotCommandGoal.ACTION_MOVE_NAME, args[0]]
    goal = RobotCommandGoal(target=target)
    action_client.send_goal(goal, done_cb=cb_reset_done)

# call_robot_to_calibrate()
# call_robot_to_move_posename(RobotCommandGoal.POSENAME_HOME)
call_robot_to_move_posename(RobotCommandGoal.POSENAME_STOW)
# call_robot_to_move_posename(3)
# call_robot('move_cell', [0, 0, 0, 0,])

error: the error is INVALID POSE


In [ ]:
import cgras_robot.msg

def cgras_cb_calibrate_done(status, result):
    if status == GoalStatus.SUCCEEDED:
        rospy.loginfo(f'success: the location apriltag found {result.location}')
    elif status == GoalStatus.ABORTED:
        rospy.loginfo(f'error: the error is {result.error}')   

def cgras_cb_move_done(status, result):
    if status == GoalStatus.SUCCEEDED:
        rospy.loginfo(f'success: moved to the grid coordinate')
    elif status == GoalStatus.ABORTED:
        rospy.loginfo(f'error: the error is {result.error}')   

def cgras_cb_reset_done(status, result):
    if status == GoalStatus.SUCCEEDED:
        rospy.loginfo(f'success: reset to the desired pose')
    elif status == GoalStatus.ABORTED:
        rospy.loginfo(f'error: the error is {result.error}')              


def cgras_call_robot(event, *args):
    if event == 'calibrate':
        action_client = actionlib.SimpleActionClient('/cgras/robot/calibrate', 
                                                            cgras_robot.msg.CalibrateAction)
        action_client.wait_for_server()
        goal = cgras_robot.msg.CalibrateGoal()
        action_client.send_goal(goal, done_cb=cgras_cb_calibrate_done)
            
    elif event == 'move':
        action_client = actionlib.SimpleActionClient('/cgras/robot/move', 
                                                            cgras_robot.msg.MoveAction)
        tile_x = int(args[0])
        tile_y = int(args[1])
        grid_x = int(args[2])
        grid_y = int(args[3])
        action_client.wait_for_server()
        goal = cgras_robot.msg.MoveGoal(tile_x=tile_x, tile_y=tile_y, grid_x=grid_x, grid_y=grid_y)
        action_client.send_goal(goal, done_cb=cgras_cb_move_done)

    elif event == 'reset':
        action_client = actionlib.SimpleActionClient('/cgras/robot/reset', 
                                                            cgras_robot.msg.ResetAction)
        pose = args[0]
        action_client.wait_for_server()
        goal = cgras_robot.msg.ResetGoal(pose=pose)
        action_client.send_goal(goal, done_cb=cgras_cb_reset_done)          
            